# Bernstein-Vazirani and Simon algorithms

In [32]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.extensions import UnitaryGate
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit_aer import *
import numpy as np



# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
# from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
# service = QiskitRuntimeService(channel="ibm_quantum")

## Bernstein-Vazirani

You can use the circuit defined for Deutsch-Josza algorithm. Two adjustments must be made:
1) The oracle must be adapted to depend on a bitstring c (the function f in Deutsch oracle is f(x)=c.x -inner product mod 2-)
2) The output is the whole bitstring measured by the circuit (instead of just checking if the measure is 0000...0).

In [33]:
def ket_reg(n,x):
    if x < 0 or x > 2**n-1:
        return(-1)
    output = [[0] for i in range(x)]+[[1]]+ [[0] for i in range(2**n-x-1)]
    return(np.matrix(output))

In [34]:
def bra_reg(n,x):
    return(ket_reg(n,x).H)

In [35]:
def Oracle(n,f):
    unitary = np.matrix([[0 for i in range(2**(n + 1))] for j in range(2**(n + 1))])
    
    for b in range(2 ** n):
        if f(b)==0:
            unitary += np.kron(np.matrix([[1,0],[0,1]]),ket_reg(n, b) @ bra_reg(n, b)) ## kron is the kronecker product, @ is the matrix product.
        else:
            unitary += np.kron(np.matrix([[0,1],[1,0]]),ket_reg(n, b) @ bra_reg(n, b))
    # print(unitary)
    return(UnitaryGate(unitary,label='U_f'))
    

In [36]:
def f(n, c):
    c = f'{c:0{n}b}'
    def output(x):
        x = f'{x:0{n}b}'
        res = 0
        for i in range(n):
            res += (int(x[i]) * int(c[i]))
        return res % 2
        
    return(output)
    

In [37]:
def Bernstein_Vazirani(n, O):
    qub = []
    qc = QuantumCircuit(0,0 )
    cr = ClassicalRegister(n, "res")
    qc.add_register(cr)
    
    for i in range(n):
        qi = QuantumRegister(1, i)
        qub.append(qi)
        qc.add_register(qub[i])
        qc.h(qub[i])
    
    minus = QuantumRegister(1, "-");
    qc.add_register(minus)
    qc.x(minus)
    qc.h(minus)
    qub.append(minus)
    
    qc.append(O, qub)
    
    
    for i in range(len(qub) - 1) :
        qc.h(qub[i])
        # qc.measure(qub[i], res)
        
    qc.measure(list(range(n)),cr)
        
    # qc.draw('mpl')
        
    aer_sim = Aer.get_backend('aer_simulator')
    result = aer_sim.run(qc, shots=1, memory=True).result()
    measured_bits = result.get_memory()
    
    return measured_bits


In [38]:
g = f(3,5)
g(5) # the mask s = 101 which is 5 in base 10

0

In [39]:
print("The mask found by the algorithm is", Bernstein_Vazirani(3, Oracle(3,g))) 

The mask found by the algorithm is ['101']


## Simon

The following function defines, given a bitstring mask s, a random $f:\{ 0,1 \}^n\to \{0,1\}^n$ such that $f(x)=f(y)\iff x=y\oplus s$.

In [40]:
def Simon_f(s:str):
    n = len(s)
    per = np.random.permutation(n)
    def f(x):
        out1 = min (x, x ^ int(s,2))
        out1 = f'{out1:0{n}b}'
        out2 = ""
        for i in range(n):
            out2 = out2 + out1[per[i]]
        return int(out2, 2)
    return f   

In [31]:
g = Simon_f("1001")
print(g(3),g(10)) #the output should be the same random value of size 4 because 0011 XOR 1010 = 1001 

9 9


Now you must construct an XOR oracle associated to a function $f:\{ 0,1 \}^n\to \{0,1\}^n$ of $n$ bits.

Using the explicit expression of the oracle in terms of output of any input, $$U_f |x,y\rangle = |x,y\oplus f(x)\rangle$$, then
$$U_f = \sum_{x\in \{0,1\}^n} \sum _{y\in \{0,1\}^n} |x,y\oplus f(x) \rangle\langle x,y|.$$

In [41]:
def Oracle_S(n,f):
    unitary = np.matrix([[0 for i in range(2**(2 * n))] for j in range(2**(2 * n))])
    
    for i in range(2**(n)):
        for j in range(2**(n)):
            unitary += np.kron(ket_reg(n, j^f(i)), ket_reg(n, i)) @ np.kron(bra_reg(n, j), bra_reg(n, i))   
                       
    return(UnitaryGate(unitary,label='U_f'))

In [42]:
print(Oracle_S(4, g))

Instruction(name='unitary', num_qubits=8, num_clbits=0, params=[array([[1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 1.+0.j]])])


Then, you have to put the oracle inside of the full Simon's algorithm circuit and run it with $n+10$ shots (for error probability smaller than $10^{-3}$).

You can use something as ```result = AerSimulator().run(quantum_circuit_name, shots=n+10, memory=True).result()```and access the measures with ```result.get_memory()```

In [44]:
def Simon(n, O):   
    qub = []
    qc = QuantumCircuit(0,0 )
    cr = ClassicalRegister( n, "res")
    qc.add_register(cr)
    
    for i in range(n):
        qi = QuantumRegister(1, i)
        qub.append(qi)
        qc.add_register(qub[i])
        qc.h(qub[i])
    
    for i in range(n, 2 * n):
        qi = QuantumRegister(1, i)
        qub.append(qi)
        qc.add_register(qub[i])
    
    qc.append(O, qub)
    
    
    for i in range(len(qub)) :
        qc.h(qub[i])
    
    qc.barrier()
    
    qc.measure(list(range(n)),cr)
    
    aer_sim = Aer.get_backend('aer_simulator')
    result = aer_sim.run(qc, shots=n + 10, memory=True).result() #n queries + C, where C = 10 in order to get 
    # at least n linear independent pairs
    measured_bits = result.get_memory()
    
    return measured_bits

In [51]:
print("The values of z measured are \n", Simon(4, Oracle_S(4,g)))

The values of z measured are 
 ['0000', '0000', '0000', '1101', '0000', '0000', '1101', '0000', '0000', '1101', '0000', '0010', '0000', '0000']


In [47]:
measurements = Simon(4, Oracle_S(4, g))
print(measurements)
# the system of equations formed by this measurement should be solved (in binary) and the solution is s, the mask

['0000', '1010', '0000', '1101', '1010', '0101', '1101', '0101', '0000', '0010', '1010', '0000', '0000', '0000']


Finally, we need a procedure that classically solves the linear system (mod 2) posed by the $n+10$ measures. We saw that that is in turn equivalent to compute a nonzero vector of the nullspace of the matrix given by all the measures. You can use the ```galois``` library inside ```numpy```

Somthing along the lines of
``` import(galois)
    matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)
    null_space = galois.GF(2)(matrix).null_space()
    display(null_space)


In [39]:
import numpy
import galois
matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)
null_space = galois.GF(2)(matrix).null_space()
display(null_space)

Traceback (most recent call last):
  Cell In[39], line 2
    import galois
ModuleNotFoundError: No module named 'galois'

Use %tb to get the full traceback.
